# NEST implementation of the `astrocyte` models

#### Han-Jia Jiang, 2023-02-23

This notebook provides a reference solution for the the _astrocyte_ model in NEST.

## The problem

The equations governing the evolution of the astrocyte model are

$$\left\lbrace\begin{array}{rcl}
    \frac{d[Ca^{2+}](t)}{dt} &=& J_{leak}(t) - J_{pump}(t)  + J_{channel}(t) \\
    J_{leak}(t) &=& r_{ER,cyt} \cdot r_L \cdot \left( [Ca^{2+}]_{ER}(t) - [Ca^{2+}](t) \right) \\
    J_{channel}(t) &=& r_{ER,cyt} \cdot v_{IP_3R} \cdot m_{\infty}(t)^3 \cdot n_{\infty}(t)^3\cdot h_{IP_3R}(t)^3 \cdot \left([Ca^{2+}]_{ER} - [Ca^{2+}](t)\right) \\
    m_{\infty}(t) &=& \frac{[IP_3](t)}{[IP_3](t) + K_{IP_{3,1}}} \\
    n_{\infty}(t) &=& \frac{[Ca^{2+}](t)}{[Ca^{2+}](t) + K_{act}} \\    
    [Ca^{2+}]_{ER}(t) &=& \frac{Ca_{tot}-[Ca^{2+}](t)}{r_{ER,cyt}} \\
    \frac{dh_{IP_3}(t)}{dt} &=& \alpha_{h_{IP_3}}(t) \cdot (1-h_{IP_3}(t)) - \beta_{h_{IP_3}}(t) \cdot h_{IP_3} (t) \\
    \alpha_{h_{IP_3}}(t) &=& r_{IP_3} \cdot K_{inh} \cdot  \frac{[IP_3](t) + K_{IP_3,1}}{[IP_3](t)+K_{IP_3,2}} \\
    \beta_{h_{IP_3}}(t) &=& r_{IP_3R} \cdot [Ca^{2+}](t) \\
    \frac{d[IP_3](t)}{dt} &=& \frac{[IP_3 ]^{*} - [IP_3](t)}{\tau_{IP_3}} + r_{IP_3} \cdot \left(J^{spike}(t)\right) \\
    J^{spike}(t) &=& \sum_{s} w_{s} \cdot \delta(t-t_{s}) \\
\end{array}\right.$$


## Technical details and requirements

### Implementation of the functions

* The reference solution is implemented through [scipy](http://www.scipy.org/).

### Requirements

To run this notebook, you need:

* [numpy](http://www.numpy.org/) and [scipy](http://www.scipy.org/)


In [1]:
import numpy as np
from scipy.integrate import odeint
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (15, 6)

## Reference solution
### Right hand side function

In [2]:
def rhs(y, _, p):
    '''
    Implementation of astrocyte dynamics
    
    Parameters
    ----------
    y : list
        Vector containing the state variables [IP3, Ca, h_IP3R]
    _ : unused var
    p : Params instance
        Object containing the neuronal parameters.
        
    Returns
    -------
    dCa : double
        Derivative of Ca
    dIP3 : double
        Derivative of IP3
    dh_IP3R : double
        Derivative of h_IP3R
    '''
    IP3 = y[0]
    Ca = y[1]
    h_IP3R = y[2]

    alpha = p.k_IP3R*p.Kd_inh*(IP3+p.Kd_IP3_1)/(IP3+p.Kd_IP3_2)
    beta = p.k_IP3R*Ca
    Ca_ER = (p.Ca_tot - Ca)/p.ratio_ER_cyt
    m_inf = IP3/(IP3+p.Kd_IP3_1)
    n_inf = Ca/(Ca + p.Kd_act)
    J_ch = p.ratio_ER_cyt*p.rate_IP3R*((m_inf*n_inf*h_IP3R)**3)*(Ca_ER - Ca)
    J_pump = p.rate_SERCA*(Ca**2)/(p.Km_SERCA**2 + Ca**2)
    J_leak = p.ratio_ER_cyt*p.rate_L*(Ca_ER - Ca)

    dCa = J_ch - J_pump + J_leak
    dIP3 = (p.IP3_0 - IP3)/p.tau_IP3
    dh_IP3R = alpha*(1-h_IP3R) - beta*h_IP3R
    
    return dIP3, dCa, dh_IP3R


### Complete model

In [3]:
def scipy_astrocyte(p, f, simtime, dt):
    '''
    Complete astrocyte model using scipy `odeint` solver.
    
    Parameters
    ----------
    p : Params instance
        Object containing the astrocyte parameters.
    f : function
        Right-hand side function
    simtime : double
        Duration of the simulation (will run between
        0 and tmax)
    dt : double
        Time increment.
        
    Returns
    -------
    t : list
        Times at which the astrocyte state was evaluated.
    y : list
        State values associated to the times in `t`
    fos : list
        List of dictionaries containing additional output
        information from `odeint`
    '''
    t = np.arange(0, simtime, dt)   # time axis
    n = len(t)                 
    y = np.zeros((n, 3))         # Ca, IP3, h_IP3R
    y[0, 0] = 1.0
    y[0, 1] = 1.0
    y[0, 2] = 1.0
    fos = []    # full output dict from odeint()
    
    # update time-step by time-step
    for k in range(1, n):
        
        # solve ODE from t_k-1 to t_k
        d, fo = odeint(f, y[k-1, :], t[k-1:k+1], (p, ), full_output=True)
        y[k, :] = d[1, :]
        fos.append(fo)
        
    return t, y, fos

### Parameters

In [4]:
astro_param = {
    'Ca_tot': 2.0,
    'IP3_0': 0.16,
    'Kd_act': 0.08234,
    'Kd_inh': 1.049,
    'Kd_IP3_1': 0.13,
    'Kd_IP3_2': 0.9434,
    'Km_SERCA': 0.1,
    'ratio_ER_cyt': 0.185,
    'incr_IP3': 5.0,
    'k_IP3R': 0.0002,
    'rate_L': 0.00011,
    'tau_IP3': 7142.0,
    'rate_IP3R': 0.006,
    'rate_SERCA': 0.0009,
}

class Params:
    '''
    Class giving access to the astrocyte
    parameters.
    '''
    def __init__(self):
        self.params = astro_param
        self.Ca_tot = astro_param["Ca_tot"]
        self.IP3_0 = astro_param["IP3_0"]
        self.Kd_act = astro_param["Kd_act"]
        self.Kd_inh = astro_param["Kd_inh"]
        self.Kd_IP3_1 = astro_param["Kd_IP3_1"]
        self.Kd_IP3_2 = astro_param["Kd_IP3_2"]
        self.Km_SERCA = astro_param["Km_SERCA"]
        self.ratio_ER_cyt = astro_param["ratio_ER_cyt"]
        self.incr_IP3 = astro_param["incr_IP3"]
        self.k_IP3R = astro_param["k_IP3R"]
        self.rate_L = astro_param["rate_L"]
        self.tau_IP3 = astro_param["tau_IP3"]
        self.rate_IP3R = astro_param["rate_IP3R"]
        self.rate_SERCA = astro_param["rate_SERCA"]
    
p = Params()

### Simulate the implementation

In [5]:
# Parameters of the simulation
simtime = 100.
resolution = 0.01

t, y, fos = scipy_astrocyte(p, rhs, simtime, resolution)
data = np.concatenate((np.array([t]).T, y), axis=1)

np.savetxt(
    'test_astrocyte.dat', data[1:, :],
    header='Times IP3 Ca h_IP3R')


-----------------------------
### License

This file is part of NEST. Copyright (C) 2004 The NEST Initiative

NEST is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 2 of the License, or (at your option) any later version.

NEST is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.